# airspace_renderer Example
The airspace geometries shown in this example are exactly that - examples!

Do not use any of this data for operational reasons. Duh.

In [1]:
import csv
import logging

import folium
import geopandas

from airspace_renderer.airspace import parse_polygon, BorderProvider
from airspace_renderer.crs import CRS_WGS84
from example_border_provider import border_provider

log = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [2]:
def parse_airspaces(path: str, border_provider: BorderProvider) -> geopandas.GeoDataFrame:
    with open(path, 'r') as airspace_csv:
        reader = csv.DictReader(airspace_csv)
        name = []
        geometry = []
        for row in reader:
            name.append(row["name"])
            geometry.append(parse_polygon(row["geometry"], border_provider))
    return geopandas.GeoDataFrame({
        "name": name,
        "geometry": geometry
    }, crs=CRS_WGS84)

In [3]:
gdf_airspaces = parse_airspaces("./example-data/airspaces.csv", border_provider)
gdf_restricted_areas = parse_airspaces("./example-data/restricted_areas.csv", border_provider)
gdf_danger_areas = parse_airspaces("./example-data/danger_areas.csv", border_provider)
gdf_fir = parse_airspaces("./example-data/fir.csv", border_provider)
m = folium.Map(location=(47.0, 8.5), zoom_start=8)

for border_name in ["CH", "LI"]:
    border_points = (
        border_provider.get_border(border_name).coords
    )
    for border_point in border_points:
        folium.CircleMarker(
            (border_point[1], border_point[0]),
            radius=5,
            color="magenta",
            stroke=False,
            fill=True,
            fill_opacity=0.6,
            opacity=1,
        ).add_to(m)

gdf_airspaces.explore(m=m, color="blue")
gdf_restricted_areas.explore(m=m, color="orange")
gdf_danger_areas.explore(m=m, color="yellow")
gdf_fir.explore(m=m, color="purple", style_kwds={"fill": False})